# CellChat

In [ ]:
options(warn=-1, stringsAsFactors=FALSE)

In [ ]:
library_load <- suppressMessages(
    
    list(
        
        # CellTalk
        library(CellChat), 
        
        # Seurat 
        library(Seurat), 
        
        # Data 
        library(tidyverse), 
        
        # Plotting 
        library(ComplexHeatmap), 
        library(circlize), 
        library(viridis), 
        library(ggplotify), 
        library(ggrepel), 
        library(cowplot)
        
    )
)

In [ ]:
random_seed <- 42
set.seed(random_seed)

In [ ]:
ht_opt$message=FALSE # ComplexHeatmap 

In [ ]:
# Set working directory to project root
setwd("/research/peer/fdeckert/FD20200109SPLENO")

In [ ]:
# Source files
source("plotting_global.R")
source("bin/cellchat.R")

In [ ]:
# Plotting Theme
ggplot2::theme_set(theme_global_set(1)) # From project global source()

# Parameter and files 

In [ ]:
so_file <- "data/BSA_0355_SM01_10x_SPLENO/ANALYSIS/pp.rds"

In [ ]:
cache_cellchat <- TRUE

# Import 

In [ ]:
so <- readRDS(so_file)

In [ ]:
# Filter genes
cnt <- GetAssayData(so, assay="RNA", slot="counts")
cnt <- cnt[rowSums(cnt>=1)>=3, ]

so <- CreateSeuratObject(counts=cnt, meta.data=so@meta.data)
    
# Get normalized counts 
so <- NormalizeData(so, normalization.method="LogNormalize", verbose=FALSE)

## Pre-process cellchat objects for each infection condition 

In [ ]:
cellchat_pp <- function(so, group_by) {
    
    # Get normalized data 
    nrm <- GetAssayData(so, assay="RNA", slot="data")
    
    # Get meta data 
    meta <- so@meta.data
    meta <- droplevels(meta)
    
    cellchat <- createCellChat(object=nrm, meta=meta, group.by=group_by)
    cellchat <- setIdent(cellchat, ident.use=group_by)
    cellchat@DB <- CellChatDB.mouse # Add DB slot with interaction, complex, cofactor, and geneInfo data frames 
    cellchat <- subsetData(cellchat) # Subset data for signaling genes 
    # cellchat <- identifyOverExpressedGenes(cellchat, thresh.p=1) # Identify DE signaling genes stored in object@var.features. Disabled for now since I want to keep all genes for analysis.
    cellchat@var.features <- list(features=rownames(cellchat@data))
    cellchat <- identifyOverExpressedInteractions(cellchat) # Filter interaction based on over expressed genes. Adds @LR$LRsig which is a pruned persion of cellchat@DB$interaction
    cellchat <- smoothData(cellchat, adj = PPI.mouse) # Project protein data 
    cellchat <- computeCommunProb(cellchat, raw.use=FALSE, population.size=FALSE, type="truncatedMean", trim=0.01) # Using the information from DB compute interaction strength (probability) in object@net$prob and associated p-values in object@net$pval. 
    cellchat <- computeCommunProbPathway(cellchat) # Summarise L-R interaction on pathway level. Return object@netP$prob is the pathway probability and significant pathways are stored in object@netP$pathways
    cellchat <- aggregateNet(cellchat) # Updates object@net with counts and weights 
    cellchat <- netAnalysis_computeCentrality(cellchat) 
    
    return(cellchat)
    
}

In [ ]:
if(!cache_cellchat) {
    
    cellchat_main_baseline <- cellchat_pp(subset(so, subset=infection=="Baseline"), group_by="leiden_cell_type_main")
    saveRDS(cellchat_main_baseline, "data/BSA_0355_SM01_10x_SPLENO/ANALYSIS/cellchat/cellchat_baseline.rds")

} else {
    
    cellchat_main_baseline <- readRDS("data/BSA_0355_SM01_10x_SPLENO/ANALYSIS/cellchat/cellchat_baseline.rds")
    
}

In [ ]:
if(!cache_cellchat) {
    
    cellchat_main_cpg <- cellchat_pp(subset(so, subset=infection=="CpG"), group_by="leiden_cell_type_main")
    saveRDS(cellchat_main_cpg, "data/BSA_0355_SM01_10x_SPLENO/ANALYSIS/cellchat/cellchat_cpg.rds")

} else {
    
    cellchat_main_cpg <- readRDS("data/BSA_0355_SM01_10x_SPLENO/ANALYSIS/cellchat/cellchat_cpg.rds")
    
}

# Merge CellChat data objects 

In [ ]:
cellchat_main_list <- list(Baseline=cellchat_main_baseline, CpG=cellchat_main_cpg)
cellchat_main <- mergeCellChat(cellchat_main_list, add.names=names(cellchat_main_list))

In [ ]:
cellchat_main@meta$infection <- factor(cellchat_main@meta$infection, levels=c("Baseline", "CpG"))

# Heatmap of number of interactions 

In [ ]:
options(repr.plot.width=20, repr.plot.height=20)

hm_1 <- hm_net(cellchat=cellchat_main_baseline, slot="count", title="Interaction count Baseline") %>% ggplotify::as.ggplot()
hm_2 <- hm_net(cellchat=cellchat_main_cpg, slot="count", title="Interaction count CpG") %>% ggplotify::as.ggplot()

hm_3 <- hm_net(cellchat=cellchat_main_baseline, slot="weight", title="Interaction weight Baseline") %>% ggplotify::as.ggplot()
hm_4 <- hm_net(cellchat=cellchat_main_cpg, slot="weight", title="Interaction weight CpG") %>% ggplotify::as.ggplot()

hm_1 + hm_2 + hm_3 + hm_4 + patchwork::plot_layout(nrow=2)

# Compare pathways between infection groups

In [ ]:
options(repr.plot.width=10, repr.plot.height=35)

gg1 <- rankNet(cellchat_main, mode="comparison", stacked=TRUE, do.stat=TRUE, comparison=c(1, 2)) + 
    scale_fill_manual(values=unlist(color$infection), breaks=c("CpG", "Baseline")) + 
    theme_global_set(1) + 
    theme(axis.text.y=element_text(size=16, color="black"), legend.title=element_blank()) 

gg1

# Plot gene contribution to pathway

In [ ]:
# options(repr.plot.width=10, repr.plot.height=5)

# parilr <- data.frame(interaction_name=c("CCL2_ACKR1", "CCL5_ACKR1", "CCL17_ACKR1"))
# sources <- c("ncMo", "cMo", "RPM")
# targets <- c("MEP", "ProEB", "EB I", "EB II", "EB III")

# plot_list <- list()

# plot_list[[1]] <- netVisual_chord_gene(cellchat_main_baseline, lab.cex=1, pairLR.use=parilr, thresh=Inf, show.legend=FALSE, sources.use=sources, targets.use=targets, small.gap=3)
# plot_list[[2]] <- netVisual_chord_gene(cellchat_main_cpg, lab.cex=1, pairLR.use=parilr, thresh=Inf, show.legend=FALSE, sources.use=sources, targets.use=targets, small.gap=3)

# cowplot::plot_grid(plotlist=plot_list, ncol=2, nrow=1)

In [ ]:
# pdf("result/figures/cellchat_ackar1_net_chord_gene_Baseline.pdf", width=5, height=5)
# replayPlot(plot_list[[1]])
# dev.off()

# pdf("result/figures/cellchat_ackar1_net_chord_gene_cpg.pdf", width=5, height=5)
# replayPlot(plot_list[[2]])
# dev.off()